In [1]:
import requests
from urllib.parse import quote
import base64
import re

In [82]:
class TippiLink:
    def __init__(self, username, password, host="192.168.0.1"):
        self._username = username
        self._password = password
        self._host = host

    def _get_headers(self):
        headers = {
            'Referer': 'http://%s/userRpm/MenuRpm.htm' % self._host,
            'Authorization': self._auth_header(),
        }
        return headers

    def _auth_header(self):
        auth_bytes = bytes(self._username+':'+self._password, 'utf-8')
        auth_b64_bytes = base64.b64encode(auth_bytes)
        auth_b64_str = str(auth_b64_bytes, 'utf-8')

        auth_str = quote('Basic {}'.format(auth_b64_str))
        return auth_str
    
    def _get_mac_from_page(self, pageName):

        response = requests.get('http://%s/userRpm/%s.htm' % (self._host, pageName), headers=self._get_headers())

        content = response.content
        
        ini = 'hostList = new Array('
        fin = ');'

        texto = str(content)[str(content).index(ini)+len(ini)+2:]
        arrTx = texto[:texto.index(fin)].split("\"")
        
        arr = [l for l in arrTx if len(l.split('-')) == 6]
        
        return arr
        #return texto

In [83]:
tl = TippiLink("luis", "AmE3024OT1", "192.168.66.32")


In [ ]:
response = requests.get('http://%s/userRpm/%s.htm' % (tl._host, 'StatusRpm.htm'), headers=tl._get_headers())

content = response.content

In [92]:
ini = 'wlanHostPara = new Array('
fin = ');'

response = requests.get('http://%s/userRpm/%s.htm' % (tl._host, 'WlanStationRpm.htm'), headers=tl._get_headers())
content = response.content

lista = str(content)[str(content).index(ini)+len(ini):str(content).index(fin)].replace('\\n', '').split(',')
print('Conex = ', lista[5])

Conex =   16


In [93]:
page = '1'
macs = []

while True:
    arr = tl._get_mac_from_page("WlanStationRpm.htm?Page="+page)
    print(page, len(macs), len(arr))
    page = str(int(page) + 1)

    if set(arr).issubset(set(macs)):
        break
    else:
        macs.extend(arr)

mac_address = list(set(macs))

mac_address = [mac.replace('-', ':').lower() for mac in mac_address]

len(mac_address)

1 0 8
2 8 8
3 16 5
4 21 5


21

In [94]:
mac_address

['bc:25:e0:92:da:4c',
 '60:14:66:d1:e5:5c',
 'bc:44:86:90:fc:ca',
 '88:79:7e:11:cb:33',
 '00:91:10:74:01:74',
 '0c:ce:f6:6d:8f:3f',
 '64:88:ff:cf:38:9b',
 '94:27:90:5a:8b:e8',
 '02:8a:0f:01:78:59',
 '00:07:a8:98:e6:32',
 'dc:d9:16:72:57:93',
 'c0:d3:c0:52:f7:9b',
 'f8:1a:67:cf:ef:82',
 '90:f6:52:e1:95:4e',
 '68:5a:cf:9d:15:fa',
 '60:1d:91:c1:8a:6a',
 '00:8a:0f:05:57:66',
 '20:64:32:7b:c8:cd',
 'd0:5b:a8:05:81:f4',
 '40:3f:8c:4c:77:83',
 'a8:c8:3a:75:22:0b']

In [89]:
arr = tl._get_mac_from_page("WlanStationRpm.htm?Page=1")
len(arr)

8

In [5]:
arr = tl._get_mac_from_page("WlanStationRpm.htm?Page=2", "hostList")
len(arr)

['BC-98-DF-62-B0-D0',
 'E8-D4-E0-A7-0B-43',
 '7C-D6-61-95-38-5C',
 'BC-44-86-90-FC-CA',
 'BC-98-DF-3D-79-8F',
 'B4-1C-30-E5-70-65',
 '40-3F-8C-4C-77-83',
 'F0-EE-10-14-8C-4E']

In [8]:
arr = tl._get_mac_from_page("WlanStationRpm.htm?Page=3", "hostList")
len(arr)

['BC-98-DF-43-D2-B5',
 '90-F6-52-E1-95-4E',
 'B4-39-39-0B-9F-D1',
 'FC-A6-21-36-63-DE',
 'D0-04-01-04-9E-51',
 'FC-2D-5E-CF-91-1B',
 'C8-A8-23-1F-9F-0D']

In [9]:
arr = tl._get_mac_from_page("WlanStationRpm.htm?Page=4", "hostList")
arr

['BC-98-DF-43-D2-B5',
 '90-F6-52-E1-95-4E',
 'B4-39-39-0B-9F-D1',
 'FC-A6-21-36-63-DE',
 'D0-04-01-04-9E-51',
 'FC-2D-5E-CF-91-1B',
 'C8-A8-23-1F-9F-0D']

In [15]:
len(arr) == len(set.union(set(arr1), set(arr)))

True

In [ ]:
ini = 'wlanHostPara = new Array('
fin = ');'

In [ ]:
lista = str(content)[str(content).index(ini)+len(ini):str(content).index(fin)].replace('\\n', '').split(',')
print(lista[5])

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
soup = BeautifulSoup(content, "html.parser")

In [ ]:
soup

In [ ]:
content